In [1]:
%reload_ext autoreload

In [2]:
import hydra
import timm
from omegaconf import DictConfig, OmegaConf
from pathlib import Path

In [3]:
# cd to parent directory with file .project_root
while not Path('.project-root').exists():
    %cd ..
print(f"Current working directory: {Path.cwd()}")

/Users/dgcnz/development/playground/pit_experiments
Current working directory: /Users/dgcnz/development/playground/pit_experiments


/Users/dgcnz/development/playground/pit_experiments/.venv/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [4]:
from vit_prisma.models.base_vit import HookedViT
from vit_prisma.configs.HookedViTConfig import HookedViTConfig

# The HookedViTConfig is available in the framework itself, which is the base setup for any experiment, one can customize it as per the requirements of an experiment.
config = HookedViTConfig()

# model = HookedViT(config)

In [5]:
from timm.models import VisionTransformer
from torch import nn, Tensor
import torch.nn.functional as F

In [6]:
class PixelTransformer(VisionTransformer):
    def __init__(self, img_size: int, patch_size: int, embed_dim: int, depth: int, num_heads: int, **kwargs):
        super().__init__(
            img_size=img_size,
            patch_size=patch_size,
            in_chans=1,
            num_classes=1, # will be ignored
            embed_dim=embed_dim,
            depth=depth,
            num_heads=num_heads,
            class_token=False,
            global_pool='',
            **kwargs,
        )

    def forward(self, x: Tensor):
        z = super().forward(x)
        z = z.view_as(x)
        return z

In [7]:
data_cfg_path = Path('configs/data/toy_mirror.yaml')
cfg = OmegaConf.load(data_cfg_path)
datamodule = hydra.utils.instantiate(cfg)
datamodule.setup("train")

In [8]:
from src.models.binary_grid_module import BinaryGridLightningModule
import torch

In [9]:
net=PixelTransformer(8, 1, 64, 8, 8)

In [10]:
net(torch.rand(2, 1, 8, 8)).shape

torch.Size([2, 1, 8, 8])

In [11]:
train = datamodule.train_dataloader()
x = next(iter(train))

In [12]:
x[1].shape

torch.Size([32, 1, 8, 8])

In [13]:
model = BinaryGridLightningModule(
    net=net,
    optimizer=torch.optim.Adam,
    scheduler=None,
    compile=False
)

/Users/dgcnz/development/playground/pit_experiments/.venv/lib/python3.10/site-packages/lightning/pytorch/utilities/parsing.py:208: Attribute 'net' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['net'])`.


In [14]:
import lightning as L

In [19]:
trainer = L.Trainer(max_epochs=100, accelerator="mps")

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


In [20]:
trainer.fit(model, datamodule)


  | Name         | Type             | Params | Mode 
----------------------------------------------------------
0 | net          | PixelTransformer | 404 K  | train
1 | criterion    | BCELoss          | 0      | train
2 | train_acc    | BinaryAccuracy   | 0      | train
3 | val_acc      | BinaryAccuracy   | 0      | train
4 | test_acc     | BinaryAccuracy   | 0      | train
5 | train_loss   | MeanMetric       | 0      | train
6 | val_loss     | MeanMetric       | 0      | train
7 | test_loss    | MeanMetric       | 0      | train
8 | val_acc_best | MaxMetric        | 0      | train
----------------------------------------------------------
404 K     Trainable params
0         Non-trainable params
404 K     Total params
1.617     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]